## 201710781 송영도

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install pyspark

In [3]:
import os
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [4]:
print(spark)

## 문제: 서울시 지하철 호선별 역별 유/무임 승하차 인원 분석



'서울시 지하철 호선별 역별 유/무임 승하차 인원 정보'를 csv파일로 내려받아서 분석하세요

(https://data.seoul.go.kr/dataList/OA-12251/S/1/datasetView.do?tab=S)



 (1-1) '서울시 지하철 호선별 역별 유_무임 승하차 인원 정보.csv'를 데이터프레임으로 읽고, 스키마를 출력

In [5]:
myDf = spark\
        .read.format('com.databricks.spark.csv')\
        .options(header='true', inferschema='true')\
        .load(os.path.join('/content/gdrive/MyDrive/bigdata/서울시 지하철 호선별 역별 유_무임 승하차 인원 정보.csv'))

## 답

In [6]:
myDf.printSchema()

root
 |-- 사용월: integer (nullable = true)
 |-- 호선명: string (nullable = true)
 |-- 지하철역: string (nullable = true)
 |-- 유임승차인원: integer (nullable = true)
 |-- 무임승차인원: integer (nullable = true)
 |-- 유임하차인원: integer (nullable = true)
 |-- 무임하차인원: integer (nullable = true)
 |-- 작업일자: integer (nullable = true)



* (1-2) 건수를 출력

## 답

In [7]:
myDf.count()

55171

(1-3) '호선명'별 '유임승차인원'의 순위를 매기고, 그 결과를 30줄 출력 (마지막 'rank' 컬럼)

```

## 답

In [8]:
from pyspark.sql.functions import rank
from pyspark.sql import functions as F
from pyspark.sql.window import Window
people = Window().partitionBy(['호선명']).orderBy(F.desc('유임승차인원'))
myDf.withColumn("rank",F.rank().over(people)).show(30)

+------+------+--------+------------+------------+------------+------------+--------+----+
|사용월|호선명|지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+------+--------+------------+------------+------------+------------+--------+----+
|201501| 1호선|  서울역|     1890411|     1667163|      238403|      220008|20150206|   1|
|201503| 1호선|  서울역|     1880246|     1682965|      251663|      232039|20150408|   2|
|201512| 1호선|  서울역|     1795435|      235606|     1574932|      216587|20160108|   3|
|201611| 1호선|  서울역|     1764932|      233613|     1551558|      214816|20161208|   4|
|201605| 1호선|  서울역|     1748250|      234803|     1537480|      217300|20160608|   5|
|201505| 1호선|  서울역|     1729959|     1528601|      240726|      224543|20150608|   6|
|201511| 1호선|  서울역|     1728236|     1520608|      225080|      208173|20151204|   7|
|201603| 1호선|  서울역|     1726590|      233961|     1520626|      214752|20160408|   8|
|201610| 1호선|  서울역|     1723041|      232308|     1514424|      214146|201611

* (1-4) '호선명'별 '유임승차인원'이 순위가 가장 높은 경우만 30건 출력 (마지막 'rank' 컬럼이 모두 1)

```

In [9]:
people = Window().partitionBy(['호선명']).orderBy(F.desc('유임승차인원'))

## 답

In [10]:
myDf.withColumn("rank",F.rank().over(people))\
    .filter(F.col("rank")==1)\
    .show(30)

+------+--------------+--------------------+------------+------------+------------+------------+--------+----+
|사용월|        호선명|            지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+--------------+--------------------+------------+------------+------------+------------+--------+----+
|201501|         1호선|              서울역|     1890411|     1667163|      238403|      220008|20150206|   1|
|201501|         2호선|                강남|     3266271|     3353256|      164508|      149186|20150206|   1|
|201712|         3호선|          고속터미널|     1843154|      200178|     1760221|      189058|20180103|   1|
|201512|         4호선|                혜화|     1461258|      138010|     1521596|      138133|20160108|   1|
|201612|         5호선|광화문(세종문화회관)|     1328714|      109907|     1404285|      115604|20170108|   1|
|201710|         6호선|              이태원|      580419|       36941|      680830|       38853|20171103|   1|
|201512|         7호선|      가산디지털단지|     1245731|       82588|     1243464| 

* (1-5) '호선명'별 '유임승차인원'의 zscore를 계산하고 출력. 윈도우 함수를 사용하여야 한다.

```

In [11]:
from scipy import stats
import sys

byAll = Window.rowsBetween(-sys.maxsize, sys.maxsize)

In [12]:
from pyspark.sql import functions as F
myDf = myDf.withColumn("meanBYLine", F.avg(myDf['유임승차인원']).over(byAll))

In [13]:
myDf = myDf.withColumn("sdByLine", F.stddev(myDf['유임승차인원']).over(byAll))

In [14]:
myDf = myDf.withColumn("zscore", (F.col('유임승차인원')-F.col('meanBYLine'))/F.col('sdByLine'))

## 답

In [15]:
myDf.select("호선명","유임승차인원","meanBYLine","sdByLine","zscore").show()

+------+------------+-----------------+-----------------+--------------------+
|호선명|유임승차인원|       meanBYLine|         sdByLine|              zscore|
+------+------------+-----------------+-----------------+--------------------+
| 1호선|      516353|302546.6513023146|322895.6016002317|  0.6621531778013912|
| 1호선|      119396|302546.6513023146|322895.6016002317| -0.5672132119317885|
| 1호선|      271006|302546.6513023146|322895.6016002317|-0.09768064707602991|
| 1호선|      315789|302546.6513023146|322895.6016002317| 0.04101123902604408|
| 1호선|      797960|302546.6513023146|322895.6016002317|  1.5342833604498682|
| 1호선|      222630|302546.6513023146|322895.6016002317| -0.2474999687399187|
| 1호선|      210503|302546.6513023146|322895.6016002317|-0.28505699937118173|
| 1호선|      433793|302546.6513023146|322895.6016002317|  0.4064668210011047|
| 1호선|      362377|302546.6513023146|322895.6016002317| 0.18529316720690342|
| 1호선|     1086374|302546.6513023146|322895.6016002317|  2.4274946602342413|
| 

### 문제2: 회귀분석



sklearn의 make_regression함수를 다음과 같이 사용하여 데이터를 생성하고, 문제를 푸세요.



```

from sklearn.datasets import make_regression

X, y, coef = make_regression(n_samples = 200,

                            n_features = 4,

                            n_informative = 3,

                            n_targets = 1,

                            noise = 0.0,

                            coef = True) ... [식-1]

In [16]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_samples = 200,

                            n_features = 4,

                            n_informative = 3,

                            n_targets = 1,

                            noise = 0.0,

                            coef = True)

* (2-1) gradient 방법으로 회귀식을 계산하고, 계수를 출력.

또한 [식-1]의 'coef'를 출력하고, gradient방법에서 도출된 계수와 일치하는지 비교하세요.

주의: 반복이 많으면 감점 (예를 들어, 1000 이상이면 많다고 간주할 것임)

In [17]:
import numpy as np
alpha = 0.02
theta=np.array(np.ones([X.shape[1]]))

In [18]:
def gradientDescent(x,y,theta,alpha,numIterations):
    for i in range(numIterations):
        #i+=1
        h=np.dot(x,theta)
        error=h-y
        cost=np.sum((h-y)**2)/len(x) #full MSE
        gradient=np.dot(x.T,error)*2/len(x) #w0,w1
        theta-=alpha*gradient
        if i%100==0:
            print("Iteration {0} | theta{1} Cost {2:.5f}".format(i,theta,cost))

    return theta

In [19]:
w4,w3,w2,w1=gradientDescent(X, y, theta, alpha, 800) 

Iteration 0 | theta[1.53240164 1.77322485 0.96944229 2.15280328] Cost 1239.78109
Iteration 100 | theta[14.39005865 21.36518165  0.08880483 23.59556471] Cost 0.38686
Iteration 200 | theta[1.47569948e+01 2.18678653e+01 5.76541149e-03 2.36284289e+01] Cost 0.00048
Iteration 300 | theta[1.47722244e+01 2.18832090e+01 2.87227992e-04 2.36234881e+01] Cost 0.00000
Iteration 400 | theta[1.47728851e+01 2.18837360e+01 1.28895232e-05 2.36232330e+01] Cost 0.00000
Iteration 500 | theta[1.47729135e+01 2.18837556e+01 5.53783647e-07 2.36232220e+01] Cost 0.00000
Iteration 600 | theta[1.47729147e+01 2.18837564e+01 2.33555489e-08 2.36232215e+01] Cost 0.00000
Iteration 700 | theta[1.47729147e+01 2.18837564e+01 9.77377583e-10 2.36232215e+01] Cost 0.00000


## 답


In [20]:
print("예측 결과:      %.8f  %.8f  %.8f  %.8f"%(w4,w3,w2,w1))
print("[식-1의] coef:  %.8f  %.8f  %.8f  %.8f"%(coef[0],coef[1],coef[2],coef[3]))

예측 결과:      14.77291475  21.88375640  0.00000000  23.62322147
[식-1의] coef:  14.77291475  21.88375640  0.00000000  23.62322147


* (2-2) 스파크 데이터프레임을 생성 (이후 계속 스파크로 푸세요)

In [21]:
import pandas as pd
pdDf=pd.DataFrame(X)
pdDf['y']=y

## 답

In [22]:
Df=spark.createDataFrame(pdDf)
Df.printSchema()

root
 |-- 0: double (nullable = true)
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- y: double (nullable = true)



In [23]:
Df = Df.withColumnRenamed("0", "x0")
Df = Df.withColumnRenamed("1", "x1")
Df = Df.withColumnRenamed("2", "x2")
Df = Df.withColumnRenamed("3", "x3")

In [24]:
Df.show(5)

+-------------------+--------------------+--------------------+--------------------+-------------------+
|                 x0|                  x1|                  x2|                  x3|                  y|
+-------------------+--------------------+--------------------+--------------------+-------------------+
|-0.6876101875490584|   1.372576235936033| -0.2652527579052315|-0.11988224867568169| 17.047112397237832|
|-0.9723887001843645| 0.43960266345246296|-0.39358285522628633|0.026770856506504768| -4.112443899890961|
|  1.787031221228319|  0.6154161990207595|-0.01391004386124...| -0.5094943146912125| 27.831381039780965|
|-1.0111906162480102|  1.2871784047051713|  -1.255779476136069| -0.3765050117960871| 4.3358046045742515|
|0.36918369994214356|-0.10486519011925523|   1.937964329365381| -1.0332602288164097|-21.249860173633188|
+-------------------+--------------------+--------------------+--------------------+-------------------+
only showing top 5 rows



* (2-3) 훈련, 테스트 데이터를 6:4 비율로 분리하고,

## 답

In [25]:
(trainDf, testDf) = Df.randomSplit([0.6, 0.4])

 (2-4) 회귀모델링하고 계수와 절편 출력. 

다음과 같이 실제와 예측을 10줄 출력.

In [26]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(inputCols = ["x0","x1","x2","x3"],
                                   outputCol = 'features')

In [27]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='y', maxIter=10, regParam=0.3, elasticNetParam=0.8)


In [28]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[va,lr])

In [29]:
model = pipeline.fit(trainDf)
modelTrainDf = model.transform(trainDf)
modelTestDf=model.transform(testDf)

##답

In [30]:
print("Coefficients: {}".format(model.stages[-1].coefficients)) #계수
print("Intercept: {:.3f}".format(model.stages[-1].intercept)) #y절편

Coefficients: [14.458078638434397,21.579770423974015,0.0,23.337615451036502]
Intercept: 0.082


In [31]:
modelTestDf.select("y", "prediction").show(10)

+-------------------+-------------------+
|                  y|         prediction|
+-------------------+-------------------+
| -82.79018306872197| -81.34875775052659|
| -32.04083751153449|-31.265162589321132|
| -46.39288878992134|-45.461642455554255|
|-12.079116520950851|-11.625726727483853|
|  67.81382938342324|  67.24536645119112|
| -8.472815756826918| -8.127908540892596|
| -66.44653531346476| -65.32691278294578|
| -8.743708769982721| -8.364553373328055|
|-22.299860071198182|-21.804867945111226|
| -16.91891604023228| -16.48441541903353|
+-------------------+-------------------+
only showing top 10 rows



* (2-5) 테스트 데이터에 대해 R2를 구해서 출력하세요.



In [32]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator=RegressionEvaluator(labelCol="y", predictionCol="prediction", metricName="r2")

## 답


In [33]:
f"r2: {evaluator.evaluate(modelTestDf):.8f}"

'r2: 0.99978756'

### 문제3: 텍스트 분류



다음은 영화리뷰이다.



In [34]:
import os

* (3-1) 데이터프레임을 생성하세요.

	* 영화리뷰를 파일로 작성하자. ipynb에서 직접 하거나 편리한 에디터를 사용함.

	* 리뷰는 한 줄씩 저장하며, 각각의 긍정, 부정을 구분하여 레이블을 지정하고 함께 저장함.

    * 파일로 작성하지 못하면, 이후 문제를 포기하지 말고 리스트로 만들거나 자신이 할 수 있는 방법을 통해서 만들어도 됨.

In [35]:
movieDf = spark\
    .read\
    .options(header='false', inferschema='true', delimiter='\t')\
    .csv(os.path.join('/content/gdrive/MyDrive/bigdata/3번.txt'))

In [36]:
movieDf.printSchema()

root
 |-- _c0: string (nullable = true)



In [37]:
from pyspark.sql.functions import lit
movieDf = movieDf.withColumnRenamed("_c0", "movies")
movieDf=movieDf.withColumn("label",lit(0))

## 답

In [38]:
movieDf.show(10)

+--------------------------------+-----+
|                          movies|label|
+--------------------------------+-----+
|                        ### 긍정|    0|
| - 고구마 먹은 영화 유치해서 ...|    0|
|  - 노잼이네요.. 왜 배우들이 ...|    0|
| - 제발 뻘짓 좀 그만하셨으면....|    0|
|     - 스토리가 매우 매우 아쉽다|    0|
|         - 영화 약간 이해가 안됨|    0|
|- 좋아하는 배우들인데 최근 영...|    0|
| - 그지같네 도대체 이 따위 영...|    0|
| - 스크린보다 시계를 더 많이 ...|    0|
| - 이때까지 본 영화중에 제일 ...|    0|
+--------------------------------+-----+
only showing top 10 rows



* (3-2) 불용어 제거, 불필요 단어 정리

    * 느낌표, 점, '...', 'ㅋㅋㅋ', 'ㅠ', 'ㄹㅇ' 불필요한 단어 제거

    * 불용어 제거: '그', '수', '본', '등', '때', '걸', '다', '게', '날',

	'짱', '거', '중', '더', '넘', '두', '달', '올', '뻔', '단', '개', '듯'

In [39]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="movies", outputCol="tokens")
tokDf = tokenizer.transform(movieDf)

In [40]:
tokDf.show(3)

+-------------------------------+-----+---------------------------+
|                         movies|label|                     tokens|
+-------------------------------+-----+---------------------------+
|                       ### 긍정|    0|                [###, 긍정]|
|- 고구마 먹은 영화 유치해서 ...|    0|[-, 고구마, 먹은, 영화, ...|
| - 노잼이네요.. 왜 배우들이 ...|    0|[-, 노잼이네요.., 왜, 배...|
+-------------------------------+-----+---------------------------+
only showing top 3 rows



In [41]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="tokens", outputCol="nostops")

In [42]:
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

## 답

In [43]:
_mystopwords=[u"!",u".",u"ㅋㅋㅋ",u"...",u",",u"ㄹㅇ",u"ㅠ",
              u"그",u"수",u"본",u"등",u"때",u"걸",u"다",u"게",
              u"날",u"짱",u"거",u"중",u"더",u"넘",u"두",u"달",
              u"올",u"뻔",u"단",u"개",u"듯",u"-",u"###",'-',u'[',u']',u'어우'
              ,u'왜',u'좀',u'제발',u'뻘짓']
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)
stopDf=stop.transform(tokDf)
stopDf.show()

+---------------------------------+-----+------------------------------+--------------------------------+
|                           movies|label|                        tokens|                         nostops|
+---------------------------------+-----+------------------------------+--------------------------------+
|                         ### 긍정|    0|                   [###, 긍정]|                          [긍정]|
|  - 고구마 먹은 영화 유치해서 ...|    0|   [-, 고구마, 먹은, 영화, ...|  [고구마, 먹은, 영화, 유치해...|
|   - 노잼이네요.. 왜 배우들이 ...|    0|   [-, 노잼이네요.., 왜, 배...|  [노잼이네요.., 배우들이, 출...|
|  - 제발 뻘짓 좀 그만하셨으면....|    0|   [-, 제발, 뻘짓, 좀, 그만...|    [그만하셨으면...., 이건, ...|
|      - 스토리가 매우 매우 아쉽다|    0|  [-, 스토리가, 매우, 매우,...|  [스토리가, 매우, 매우, 아쉽다]|
|          - 영화 약간 이해가 안됨|    0| [-, 영화, 약간, 이해가, 안됨]|      [영화, 약간, 이해가, 안됨]|
| - 좋아하는 배우들인데 최근 영...|    0| [-, 좋아하는, 배우들인데, ...| [좋아하는, 배우들인데, 최근,...|
|  - 그지같네 도대체 이 따위 영...|    0|  [-, 그지같네, 도대체, 이,...|  [그지같네, 도대체, 이, 따위...|
|  - 스크린보다 시계를 더 많이 ...|    0| [-, 스크린보

* (3-3) TF-IDF를 계산

In [44]:
from pyspark.ml.feature import HashingTF, IDF
hashTF = HashingTF(inputCol="nostops", outputCol="features")
hashDf = hashTF.transform(stopDf)

In [45]:
idf = IDF(inputCol="features", outputCol="idf")

idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

## 답

In [46]:
idfDf.select("nostops", "features","idf").show(3)

+------------------------------+--------------------+--------------------+
|                       nostops|            features|                 idf|
+------------------------------+--------------------+--------------------+
|                        [긍정]|(262144,[81545],[...|(262144,[81545],[...|
|[고구마, 먹은, 영화, 유치해...|(262144,[114067,1...|(262144,[114067,1...|
|[노잼이네요.., 배우들이, 출...|(262144,[14123,77...|(262144,[14123,77...|
+------------------------------+--------------------+--------------------+
only showing top 3 rows



* (3-4) 베이지안 모델을 적용해서 분류 (alpha는 1.0으로 설정)

In [47]:
from pyspark.ml.classification import NaiveBayes
nb=NaiveBayes(featuresCol='features', labelCol='label', modelType='multinomial', predictionCol='prediction',smoothing=1.0)

In [48]:
model=nb.fit(hashDf)

In [49]:
predictions=model.transform(hashDf)
predictions.select('movies','label','prediction').show(20)

+---------------------------------+-----+----------+
|                           movies|label|prediction|
+---------------------------------+-----+----------+
|                         ### 긍정|    0|       0.0|
|  - 고구마 먹은 영화 유치해서 ...|    0|       0.0|
|   - 노잼이네요.. 왜 배우들이 ...|    0|       0.0|
|  - 제발 뻘짓 좀 그만하셨으면....|    0|       0.0|
|      - 스토리가 매우 매우 아쉽다|    0|       0.0|
|          - 영화 약간 이해가 안됨|    0|       0.0|
| - 좋아하는 배우들인데 최근 영...|    0|       0.0|
|  - 그지같네 도대체 이 따위 영...|    0|       0.0|
|  - 스크린보다 시계를 더 많이 ...|    0|       0.0|
|  - 이때까지 본 영화중에 제일 ...|    0|       0.0|
|   - 나만 짜증.. 줄거리 한줄평...|    0|       0.0|
| - 로맨틱 코미디로 방향을 잡았...|    0|       0.0|
|  - 똥 싸다가 막힌 느낌 안보는...|    0|       0.0|
|  - 어우 핵노잼 영화가 왜 이렇...|    0|       0.0|
|  - 솔직히 재미없음 감독이 뭘 ...|    0|       0.0|
|- 뭘 말하고싶은지 모르겠다 아쉽다|    0|       0.0|
| - 새벽에 케이블에서 볼거 없어...|    0|       0.0|
|      - 핵노잼 억지 착한척 오졌다|    0|       0.0|
| - 오그라드는 대사는 기본에 옵...|    0|       0.0|
|- 역대급 호화캐스팅인데 스토리...|    0|       0.

* (3-5) 훈련, 테스트 데이터를 반반씩 분리하고, 훈련 후 정확성을 계산하고 출력.

In [50]:
(trainDf, testDf) = hashDf.randomSplit([0.5, 0.5])

In [51]:
train=nb.fit(trainDf)#trainDf로 훈련

In [52]:
predictions_train=train.transform(trainDf) #train예측

In [53]:
predictions_train.select('movies','label','prediction').show(20)

+---------------------------------+-----+----------+
|                           movies|label|prediction|
+---------------------------------+-----+----------+
|                         ### 부정|    0|       0.0|
|     - 감동 그 자체 스파이더맨 짱|    0|       0.0|
|   - 그냥 진짜 재밌어 진짜 잘 ...|    0|       0.0|
|  - 그지같네 도대체 이 따위 영...|    0|       0.0|
|            - 꼭 보세여 넘 재밌어|    0|       0.0|
|    - 노잼 하 2시간 반 돌려내 ...|    0|       0.0|
|             - 띵작입니다 ㄹㅇ!!!|    0|       0.0|
| - 로맨틱 코미디로 방향을 잡았...|    0|       0.0|
|- 뭘 말하고싶은지 모르겠다 아쉽다|    0|       0.0|
|  - 솔직히 재미없음 감독이 뭘 ...|    0|       0.0|
|  - 스토리 개 말도 안되는 억지...|    0|       0.0|
| - 어렸을때는 그냥 액션과 화려...|    0|       0.0|
| - 어릴적 영화관에서 처음으로 ...|    0|       0.0|
|  - 어설픈 연출력 어설픈 결말 ...|    0|       0.0|
|  - 엔드게임 이후 마블이 한 번...|    0|       0.0|
|          - 영화 약간 이해가 안됨|    0|       0.0|
|   - 올해 본 영화 중 최고의 영...|    0|       0.0|
|  - 이때까지 본 영화중에 제일 ...|    0|       0.0|
|  - 자연과 인간, 아버지와 아들...|    0|       0.0|
|  - 제발 뻘짓 좀 그만하셨으면....|   

In [54]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label")

In [55]:
predictions_test=train.transform(testDf)

## 답

In [56]:
evaluator.evaluate(predictions_train)

0.0

In [57]:
evaluator.evaluate(predictions_test)

0.0

* (3-6) 앞서 훈련한 베이지안 모델로 다음 문장을 긍정, 부정으로 분류하세요

- 고구가 먹은 느낌 킬링타음으로도 아쉽다

- 내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다

- 보다가 채널 돌림 재미없다

- 최고의 영화

- 여운이 긴 명작

- 띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요

##풀지못함


In [58]:
test=spark.createDataFrame(
    [
     ["고구가 먹은 느낌 킬링타음으로도 아쉽다"],
     ["내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다"],
     ["보다가 채널 돌림 재미없다"],
     ["최고의 영화"],
     ["여운이 긴 명작"],
     ["띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요"]
    ]
)

In [59]:
test.printSchema()

root
 |-- _1: string (nullable = true)



In [60]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="_1", outputCol="tokens")
test_tokDf = tokenizer.transform(test)